In [1]:
import sys
import geopandas as gpd
import fiona
from shapely.geometry import Polygon
from shapely.ops import transform
import math
import pyproj
sys.path.append('..')
from h3_helper import *
from s2_helper import *
from rhealpix_helper import *
from eaggr_helper import *
#from dggrid4py import DGGRIDv7, Dggs, dgselect, dggs_types

In [6]:
# Area from individual lambert azimutal projection
def get_lambert_area(geom):
    '''Area from cell's lambert azimutal projection'''
    if (-180 <= geom.centroid.x <= 180) and (-90 <= geom.centroid.y <= 90):
        proj_str = f"+proj=laea +lat_0={geom.centroid.y} +lon_0={geom.centroid.x}"
        project = pyproj.Transformer.from_crs('EPSG:4236', proj_str, always_xy=True).transform
        area = transform(project, geom).area
    else:
        print(f'invalid centroid {geom.centroid}')
        area = None
    return area

In [2]:
def get_perimeter_from_lambert(geom):
    '''Area from cell's lambert azimutal projection'''
    if (-180 <= geom.centroid.x <= 180) and (-90 <= geom.centroid.y <= 90):
        proj_str = f"+proj=laea +lat_0={geom.centroid.y} +lon_0={geom.centroid.x}"
        project = pyproj.Transformer.from_crs('EPSG:4236', proj_str, always_xy=True).transform
        perimeter = transform(project, geom).length
    else:
        print(f'invalid centroid {geom.centroid}')
        perimeter = None
    return perimeter
    

In [2]:
def get_area_perimeter_from_lambert(geom):
    '''Area from cell's lambert azimutal projection'''
    if (-180 <= geom.centroid.x <= 180) and (-90 <= geom.centroid.y <= 90):
        proj_str = f"+proj=laea +lat_0={geom.centroid.y} +lon_0={geom.centroid.x}"
        project = pyproj.Transformer.from_crs('EPSG:4236', proj_str, always_xy=True).transform
        perimeter = transform(project, geom).length
        area = transform(project, geom).area
    else:
        print(f'invalid centroid {geom.centroid}')
        perimeter = None
    return area, perimeter

In [6]:
# Area from UTM zone
def convert_wgs_to_utm(lon, lat):
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code

def get_utm_area(geom):
    '''Area from cell's UTM zone'''
    if (-180 <= geom.centroid.x <= 180) and (-90 <= geom.centroid.y <= 90):
        utm_epsg = convert_wgs_to_utm(geom.centroid.x, geom.centroid.y) 
        project = pyproj.Transformer.from_crs('EPSG:4236', f'EPSG:{utm_epsg}', always_xy=True).transform
        area = transform(project, geom).area
    else:
        print(f'invalid centroid {geom.centroid}')
        area = None
    return area

In [3]:
fiona.listlayers(r'd:\UTCloud\DGGS\grids\grids.gpkg')

['DGGRID_ISEA4H_7',
 'DGGRID_ISEA3H_9',
 'rhpix_5',
 'DGGRID_ISEA7H_5',
 'rhpix_5_anomalies',
 'DGGRID_ISEA7H_5_anomalies',
 'DGGRID_ISEA3H_9_clean',
 'DGGRID_FULLER_4T',
 'DGGRID_ISEA4D_7',
 'DGGRID_FULLER_4D',
 'H3_4',
 'EAGGR4T_7',
 'DGGRID_FULLER_7H',
 'DGGRID_ISEA4T_7',
 's2_8',
 'DGGRID_FULLER_4D_clean',
 'DGGRID_ISEA7H_5_clean',
 'DGGRID_FULLER_7H_clean',
 'rhpix_5_clean',
 's2_8_clean',
 'DGGRID_ISEA4D_7_clean']

In [3]:
grid = gpd.read_file(r'd:\UTCloud\DGGS\grids\grids.gpkg', layer='DGGRID_FULLER_4D', driver='gpkg').dropna()

In [4]:
grid['std_area'],grid['zsc'] = zip(*grid['geometry'].apply(get_area_perimeter_from_lambert))

In [5]:
grid = grid.rename(columns={'std_area':'ind_lambert_area','zsc':'lambert_perimeter'})

In [7]:
gpd.GeoDataFrame(grid).to_file(r'd:\UTCloud\DGGS\grids\grids.gpkg', layer='DGGRID_FULLER_4D', driver='GPKG')